In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [3]:
data = pd.read_csv('insurance.csv')
data.head()

age  weight  height  income_lpa  smoker     city  occupation  \
0   67   119.8    1.56        2.92   False   Jaipur     retired   
1   36   101.1    1.83       34.28   False  Chennai  freelancer   
2   39    56.8    1.64       36.64   False   Indore  freelancer   
3   22   109.4    1.55        3.34    True   Mumbai     student   
4   69    62.2    1.60        3.94    True   Indore     retired   

  insurance_premium_category  
0                       High  
1                        Low  
2                        Low  
3                     Medium  
4                       High

## feature engineering

In [6]:
data["bmi"] = data["weight"] / (data["height"] ** 2)

In [7]:
def age_group(age):
    if age < 25:
        return "young"
    elif age < 45:
        return "adult"
    elif age < 60:
        return "middle_aged"
    return "senior"

data["age_group"] = data["age"].apply(age_group)

In [8]:
def lifestyle_risk(row):
    if row["smoker"] and row["bmi"] > 30:
        return "high"
    elif row["smoker"] or row["bmi"] > 27:
        return "medium"
    else:
        return "low"

data["lifestyle_risk"] = data.apply(lifestyle_risk, axis=1)

In [9]:
tier_1_cities = ["Mumbai", "Delhi", "Bangalore", "Chennai", "Kolkata", "Hyderabad", "Pune"]
tier_2_cities = [
    "Jaipur", "Chandigarh", "Indore", "Lucknow", "Patna", "Ranchi", "Visakhapatnam", "Coimbatore",
    "Bhopal", "Nagpur", "Vadodara", "Surat", "Rajkot", "Jodhpur", "Raipur", "Amritsar", "Varanasi",
    "Agra", "Dehradun", "Mysore", "Jabalpur", "Guwahati", "Thiruvananthapuram", "Ludhiana", "Nashik",
    "Allahabad", "Udaipur", "Aurangabad", "Hubli", "Belgaum", "Salem", "Vijayawada", "Tiruchirappalli",
    "Bhavnagar", "Gwalior", "Dhanbad", "Bareilly", "Aligarh", "Gaya", "Kozhikode", "Warangal",
    "Kolhapur", "Bilaspur", "Jalandhar", "Noida", "Guntur", "Asansol", "Siliguri"
]

In [10]:
def city_tier(city):
    if city in tier_1_cities:
        return 1
    elif city in tier_2_cities:
        return 2
    else:
        return 3

data["city_tier"] = data["city"].apply(city_tier)

In [11]:
data.drop(columns=['age', 'weight', 'height', 'smoker', 'city'])[['income_lpa', 'occupation', 'bmi', 'age_group', 'lifestyle_risk', 'city_tier', 'insurance_premium_category']].sample(5)


income_lpa      occupation        bmi    age_group lifestyle_risk  \
78       14.74      freelancer  27.932798  middle_aged         medium   
22       30.00  government_job  31.771627  middle_aged         medium   
86       37.38      freelancer  18.476526        adult            low   
54        3.32         retired  21.025423       senior         medium   
55       24.93      unemployed  25.293194  middle_aged            low   

    city_tier insurance_premium_category  
78          2                       High  
22          2                        Low  
86          1                        Low  
54          2                       High  
55          1                        Low

In [12]:
X = data[["bmi", "age_group", "lifestyle_risk", "city_tier", "income_lpa", "occupation"]]
y = data["insurance_premium_category"]

In [24]:
categorical_features = ["age_group", "lifestyle_risk", "occupation", "city_tier"]
numeric_features = ["bmi", "income_lpa"]

preprocessor = ColumnTransformer(
    transformers=[
        ("num", "passthrough", numeric_features),
        ("cat", OneHotEncoder(), categorical_features),
    ]
)

model = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", RandomForestClassifier(random_state=42, n_estimators=100, max_depth=10))]
)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', 'passthrough',
                                                  ['bmi', 'income_lpa']),
                                                 ('cat', OneHotEncoder(),
                                                  ['age_group',
                                                   'lifestyle_risk',
                                                   'occupation',
                                                   'city_tier'])])),
                ('classifier',
                 RandomForestClassifier(max_depth=10, random_state=42))])

In [25]:
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        High       0.89      0.67      0.76        12
         Low       0.60      1.00      0.75         3
      Medium       0.33      0.40      0.36         5

    accuracy                           0.65        20
   macro avg       0.61      0.69      0.63        20
weighted avg       0.71      0.65      0.66        20



In [26]:
import pickle

pickle_model_path = "model.pkl"
with open(pickle_model_path, "wb") as f:
    pickle.dump(model, f)